**Creating a csv file which has the path of images and its name**

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

In [5]:
import shutil
import os

cache_path = os.path.expanduser(r"C:\Users\SAHIL\AppData\Local\Temp\tfhub_modules")
shutil.rmtree(cache_path)
print("✅ Cache cleared.")

✅ Cache cleared.


In [2]:


# Load MoveNet Thunder Model
model_url = "https://tfhub.dev/google/movenet/singlepose/thunder/4"
movenet = hub.load(model_url)



In [3]:
# Function to extract keypoints from an image
def extract_keypoints(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0).astype(np.int32)

    outputs = movenet.signatures["serving_default"](input=img)
    keypoints = outputs['output_0'].numpy().reshape(-1, 3)[:, :2]  # Extract (x, y)

    return keypoints.flatten()

# Directory where images are stored (organized by pose names)
images_dir = "images"

data, labels = [], []
for pose in os.listdir(images_dir):
    pose_path = os.path.join(images_dir, pose)
    if os.path.isdir(pose_path):
        for img_file in os.listdir(pose_path):
            img_path = os.path.join(pose_path, img_file)
            keypoints = extract_keypoints(img_path)
            data.append(keypoints)
            labels.append(pose)  # Use folder name as label

# Save dataset to CSV
df = pd.DataFrame(data)
df["label"] = labels
df.to_csv("yoga_keypoints.csv", index=False)
print("✅ Dataset Created: yoga_keypoints.csv")

✅ Dataset Created: yoga_keypoints.csv


In [11]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import json

In [12]:

# Load dataset
df = pd.read_csv("yoga_keypoints.csv")
X, y = df.iloc[:, :-1].values, df["label"].values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", RandomForestClassifier(n_estimators=200, random_state=42))
])
model.fit(X_train, y_train)

# Save trained model
joblib.dump(model, "yoga_pose_model.pkl")
print(f"✅ Model trained with {model.score(X_test, y_test) * 100:.2f}% accuracy.")
print("✅ Model saved as yoga_pose_model.pkl")


✅ Model trained with 100.00% accuracy.
✅ Model saved as yoga_pose_model.pkl


In [13]:
import cv2
import numpy as np
import joblib
import tensorflow_hub as hub
import pandas as pd
from scipy.spatial import distance
import time


In [14]:

# Load MoveNet Thunder Model
model_url = "https://tfhub.dev/google/movenet/singlepose/thunder/4"
movenet = hub.load(model_url)

# Load trained model & dataset
classifier = joblib.load("yoga_pose_model.pkl")
df = pd.read_csv("yoga_keypoints.csv")
ideal_keypoints = df.groupby("label").mean().to_dict(orient="index")

# Keypoint labels for reference
KEYPOINT_LABELS = [
    "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear", 
    "Left Shoulder", "Right Shoulder", "Left Elbow", "Right Elbow", 
    "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
    "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"
]

# Function to extract keypoints from a frame
def extract_keypoints_from_frame(frame):
    img = cv2.resize(frame, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0).astype(np.int32)

    outputs = movenet.signatures["serving_default"](input=img)
    keypoints = outputs["output_0"].numpy().reshape(-1, 3)

    # Keep only (x, y) coordinates, ignore confidence scores
    keypoints = keypoints[:, :2]

    return keypoints.flatten(), keypoints  # Flattened & full keypoints

# Enhanced function to check if enough keypoints are detected
def is_valid_pose(keypoints_array):
    # Check that we have at least 15 keypoints detected (out of 17)
    visible_joints = np.count_nonzero(keypoints_array[:, 0])
    if visible_joints < 15:
        return False
    
    # Define critical joints that must be visible
    critical_joints = {
        'shoulders': [5, 6],  # Left and right shoulder
        'hips': [11, 12],     # Left and right hip
        'knees': [13, 14],    # Left and right knee
        'ankles': [15, 16]    # Left and right ankle
    }
    
    # Check all critical joints are detected
    for joint_type, indices in critical_joints.items():
        if not all(keypoints_array[i, 0] > 0 for i in indices):
            return False
    
    # Additional check: ensure body proportions are reasonable
    # Calculate distance between shoulders
    shoulder_width = distance.euclidean(keypoints_array[5][:2], keypoints_array[6][:2])
    # Calculate distance from shoulder to hip
    torso_height = distance.euclidean(keypoints_array[5][:2], keypoints_array[11][:2])
    
    # If proportions are unrealistic (too wide or too narrow)
    if shoulder_width < 0.1 or shoulder_width > 0.5 or torso_height < 0.1:
        return False
    
    return True

# Function to calculate corrections & rating
def calculate_corrections(detected_keypoints, ideal_pose_keypoints_dict):
    corrections = []
    total_diff = 0
    num_compared = 0

    # Convert ideal keypoints to numpy array
    ideal_pose_keypoints = np.zeros(len(KEYPOINT_LABELS)*2)
    for i in range(len(KEYPOINT_LABELS)):
        ideal_pose_keypoints[i*2] = ideal_pose_keypoints_dict[f"{i*2}"]
        ideal_pose_keypoints[i*2+1] = ideal_pose_keypoints_dict[f"{i*2+1}"]

    for i, label in enumerate(KEYPOINT_LABELS):
        detected = detected_keypoints[i*2:i*2+2]
        ideal = ideal_pose_keypoints[i*2:i*2+2]

        if np.all(detected > 0) and np.all(ideal > 0):  # Only compare if both are detected
            diff = distance.euclidean(detected, ideal)
            total_diff += diff
            num_compared += 1
            if diff > 0.15:  # Adjusted threshold for normalized coordinates
                direction = ""
                if detected[0] - ideal[0] > 0.05:
                    direction += "move left "
                elif detected[0] - ideal[0] < -0.05:
                    direction += "move right "
                
                if detected[1] - ideal[1] > 0.05:
                    direction += "move up "
                elif detected[1] - ideal[1] < -0.05:
                    direction += "move down "
                
                if direction:
                    corrections.append(f"Adjust {label}: {direction.strip()}")

    if num_compared > 0:
        avg_diff = total_diff / num_compared
        rating = max(10 - int(avg_diff / 0.05), 1)  # Adjusted rating calculation
    else:
        rating = 1

    return corrections, rating

# Main execution

def main():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print(json.dumps({
            "status": "failure",
            "pose": None,
            "confidence": None,
            "rating": None,
            "feedback": None,
            "corrections": [],
            "errors": ["Could not open webcam."]
        }, indent=2))
        return

    start_time = time.time()
    frames = []
    valid_frames = 0

    while time.time() - start_time < 15:
        ret, frame = cap.read()
        if not ret:
            break

        keypoints_flat, keypoints_array = extract_keypoints_from_frame(frame)
        if is_valid_pose(keypoints_array):
            frames.append((keypoints_flat, keypoints_array))
            valid_frames += 1

        cv2.putText(frame, "Hold your yoga pose for 15 seconds", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Time remaining: {int(15 - (time.time() - start_time))}s", 
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.imshow("Yoga Pose Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if valid_frames < 5:
        result = {
            "status": "failure",
            "pose": None,
            "confidence": None,
            "rating": None,
            "feedback": None,
            "corrections": [],
            "errors": [
                "Could not detect a valid yoga pose.",
                "Make sure your full body is visible.",
                "Improve lighting or camera position.",
            ]
        }
    else:
        best_frame = max(frames, key=lambda x: np.count_nonzero(x[1][:, 0]))
        keypoints_flat, keypoints_array = best_frame
        keypoints_flat = keypoints_flat.reshape(1, -1)

        pose_probabilities = classifier.predict_proba(keypoints_flat)
        predicted_pose = classifier.predict(keypoints_flat)[0]
        max_confidence = float(np.max(pose_probabilities))

        if max_confidence < 0.85 or predicted_pose not in ideal_keypoints:
            result = {
                "status": "failure",
                "pose": predicted_pose,
                "confidence": max_confidence,
                "rating": None,
                "feedback": None,
                "corrections": [],
                "errors": ["Pose detected with low confidence."]
            }
        else:
            corrections, rating = calculate_corrections(keypoints_flat.flatten(), ideal_keypoints[predicted_pose])

            if rating >= 8:
                feedback = "Excellent form! Keep it up! 💪"
            elif rating >= 5:
                feedback = "Good attempt! Some minor adjustments needed."
            else:
                feedback = "Needs work. Focus on the corrections below."

            result = {
                "status": "success",
                "pose": predicted_pose,
                "confidence": max_confidence,
                "rating": rating,
                "feedback": feedback,
                "corrections": corrections if corrections else ["No major corrections needed! Perfect pose! 🎉"],
                "errors": []
            }

    print(json.dumps(result, indent=2))

if __name__ == "__main__":
    main()


{
  "status": "failure",
  "pose": "baddha konasana",
  "confidence": 0.535,
  "rating": null,
  "feedback": null,
  "corrections": [],
  "errors": [
    "Pose detected with low confidence."
  ]
}
